In [10]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()   # value_counts()函数可以对Series里面的每个值进行计数并且排序
    return np.nan if len(s) == 0 else s.index[0]

def processUserAppUsage():
    resTable = pd.DataFrame()
    reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    
    app_info = pd.read_csv("app_info.csv", names=['appId','category'])
    cats = list(set(app_info['category']))      # 去重APP的种类
    category2id = dict(zip(sorted(cats), range(0,len(cats))))  # {APP的种类: 编号}

    id2category = dict(zip(range(0,len(cats)), sorted(cats)))
    app_info['category'] = app_info['category'].apply(lambda x: category2id[x]) # 将APP种类转为对应的编号, 比如商务对应0
    
    
    # print(np.bincount(app_info['category'])) 可以看出有40个类的APP, 比如教育, 交通....
    
    i = 1
    
    while True:
        try:
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(1000000)
            
            # 保证上一轮读取的数据中, 最后出现的uid的所有数据条目添加到下一轮读取的数据中, 以保证在数据聚合的时候, 
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]   
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]
            
            

            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
            now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['appId','use_date'].agg(['nunique']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid')    

            now_df.columns = ['uid','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']
            # usage_cnt表示groupby('uid')['appId'].count()即30天内使用APP的次数
            # usage_appid_cnt表示groupby('uid')['appId'].agg(['nunique']即30天内使用APP的个数
            # usage_date_cnt表示groupby('uid')[use_date'].agg(['nunique']即30天内使用APP的天数

            df = df.merge(app_info, how='left', on='appId')
            now_df = now_df.merge(df.groupby('uid')['category'].nunique().to_frame(), how='left', on='uid')
            # category即30天内使用的APP的种类数
            now_df['usage_most_used_category'] = df.groupby(['uid'])['category'].transform(f)
            # usage_most_used_category即30天内使用最多的APP的种类对应的编号
            resTable = pd.concat([resTable, now_df])
            
        except StopIteration:
            break
    
    resTable.to_csv("appuseProcessed.csv",index=0)
    
    print("Iterator is stopped")

In [11]:
processUserAppUsage()

index: 1
index: 2
             uid      appId  duration   times    use_date
1000000  1845664  a00265829     414.0    15.0  2019-03-06
1000001  1845664   a0099340    1313.0    17.0  2019-03-17
1000002  1845664  a00311680     149.0     5.0  2019-03-13
1000003  1845664  a00147973     117.0     7.0  2019-03-03
1000004  1845664   a0099340    1728.0     9.0  2019-03-05
1000005  1845664   a0021880      97.0     8.0  2019-03-15
1000006  1845664   a0071100      35.0     5.0  2019-03-02
1000007  1845664   a0046242     149.0     2.0  2019-03-19
1000008  1845664  a00290037      53.0    28.0  2019-03-12
1000009  1845664  a00287085    2595.0    98.0  2019-03-04
1000010  1845664   a0099340     347.0     2.0  2019-03-19
1000011  1845664   a0099340     954.0     9.0  2019-03-02
1000012  1845664  a00157201      71.0    10.0  2019-03-08
1000013  1845664  a00289791    2518.0    61.0  2019-03-18
1000014  1845664  a00289791     437.0    15.0  2019-03-19
1000015  1845664   a0099340     405.0     6.0  2019-03

In [38]:
reader = pd.read_csv("user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
last_df = pd.DataFrame()


app_info = pd.read_csv("app_info.csv", names=['appId','category'])
dict_temp_app_info = app_info.set_index('appId').T.to_dict()  # 将dataframe转dict

category2id = {'主题个性': 0, '主题铃声': 1, '休闲娱乐': 2, '休闲游戏': 3, '休闲益智': 4, '体育射击': 5, '体育竞速': 6, '便捷生活': 7, '儿童': 8, '出行导航': 9, '动作冒险': 10, '动作射击': 11, '医疗健康': 12, '合作壁纸*': 13, '商务': 14, '图书阅读': 15, '学习办公': 16, '实用工具': 17, '影音娱乐': 18, '拍摄美化': 19, '教育': 20, '新闻阅读': 21, '旅游住宿': 22, '棋牌天地': 23, '棋牌桌游': 24, '模拟游戏': 25, '汽车': 26, '电子书籍': 27, '益智棋牌': 28, '社交通讯': 29, '策略游戏': 30, '经营策略': 31, '网络游戏': 32, '美食': 33, '表盘个性': 34, '角色扮演': 35, '角色游戏': 36, '购物比价': 37, '运动健康': 38, '金融理财': 39}




i = 1

while True:
    try:
        print("index: {}".format(i))
        i+=1
        df = reader.get_chunk(1000000)

        df = pd.concat([last_df, df])
        idx = df.shape[0]-1
        last_user = df.iat[idx,0]
        while(df.iat[idx,0]==last_user):
            idx-=1
        last_df = df[idx+1:]
        df = df[:idx+1]
        
        # 将appId替换为APP类的Id
        df['appId'] = df['appId'].apply(lambda x : 40 if x not in dict_temp_app_info else category2id[dict_temp_app_info[x]['category']])
        
        now_df = pd.DataFrame()
        now_df['uid'] = df['uid'].unique()
        #print(df.groupby(['uid', 'appId'])['times'].sum().reset_index())
        now_df = now_df.merge(df.groupby(['uid', 'appId'])['times'].sum().reset_index(), how='left', on='uid')
        
        #print(now_df)
        
        now_df.to_csv("app_category_time_Table.csv",index=0, mode='a',header=0)

    except StopIteration:
        break

d:\workenvironment\python3.7\lib\site-packages\ipykernel_launcher.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89
index: 90
index: 91
index: 92
index: 93
index: 94
index: 95
index: 96
index: 97
index: 98
index: 99
index: 100
index: 1

In [3]:
from sklearn.preprocessing import OneHotEncoder

reader = pd.read_csv("app_category_duration_Table.csv", names=['uid','app_categoryId', 'app_category_duration'], header=0, iterator=True)
last_df = pd.DataFrame()    
    
index = 1

while True:
    try:
        print("index: {}".format(index))
        index+=1
        df = reader.get_chunk(1000000)
        df = pd.concat([last_df, df])
        idx = df.shape[0]-1
        last_user = df.iat[idx,0]
        while(df.iat[idx,0]==last_user):
            idx-=1
        last_df = df[idx+1:]
        df = df[:idx+1]
        
        for i in range(41):
            temp = {'uid': 1000364,'app_categoryId' : i, 'app_category_duration' : 0}
            df = df.append(temp, ignore_index=True)
        
        temp = df['app_categoryId']
        df = pd.get_dummies(df, columns=['app_categoryId'])
        df['app_categoryId'] = temp

        def f(x):
            x['app_categoryId_' + str(int(x['app_categoryId']))] = x['app_category_duration']
            return x
            
        new_df = df.apply(f, axis = 1)
        new_df = new_df.drop('app_category_duration', axis=1)
        new_df = new_df.drop('app_categoryId', axis=1)
        
        new_df.to_csv("app_category_duration.csv",index=0, mode='a',header=0)
        
    except StopIteration:
        break    

index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24


In [9]:
from sklearn.preprocessing import OneHotEncoder

reader = pd.read_csv("app_category_time_Table.csv", names=['uid', 'app_categoryId', 'times'], iterator=True)
last_df = pd.DataFrame()    
    
index = 1

while True:
    try:
        print("index: {}".format(index))
        index+=1
        df = reader.get_chunk(1000000)
        df = pd.concat([last_df, df])
        idx = df.shape[0]-1
        last_user = df.iat[idx,0]
        while(df.iat[idx,0]==last_user):
            idx-=1
        last_df = df[idx+1:]
        df = df[:idx+1]
        
        for i in range(41):
            temp = {'uid': 1000364,'app_categoryId' : i, 'times' : 0}
            df = df.append(temp, ignore_index=True)
        
        temp = df['app_categoryId']
        df = pd.get_dummies(df, columns=['app_categoryId'])
        df['app_categoryId'] = temp


        def f(x):
            x['app_categoryId_' + str(int(x['app_categoryId']))] = x['times']
            return x
        
        tqdm.pandas(desc="my bar！")
        new_df = df.progress_apply(f, axis = 1)
        new_df = new_df.drop('times', axis=1)
        new_df = new_df.drop('app_categoryId', axis=1)
        
        new_df.to_csv("app_category_time.csv",index=0, mode='a',header=0)
        
    except StopIteration:
        break    

index: 1


my bar！: 100%|█████████████████████████████████████████████████████████████| 1000031/1000031 [01:43<00:00, 9622.75it/s]


index: 2


my bar！: 100%|█████████████████████████████████████████████████████████████| 1000037/1000037 [01:41<00:00, 9847.84it/s]


index: 3


...

index: 22


my bar！: 100%|█████████████████████████████████████████████████████████████| 1000043/1000043 [01:40<00:00, 9902.41it/s]


index: 23


my bar！: 100%|███████████████████████████████████████████████████████████████| 632729/632729 [01:04<00:00, 9861.16it/s]


index: 24


In [12]:
import pandas as pd
import numpy as np
import lightgbm as lgb       # 基于决策树的梯度Boosting框架, 比Xgboost占用内存少, 速度快, 准确率更高
from tqdm import tqdm

In [13]:
pd.set_option('display.max_columns',None)     # 设置即使列数很多, 也不省略表示

In [14]:
#读取数据
age_train = pd.read_csv("age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv("age_test.csv", names=['uid'])
user_basic_info = pd.read_csv("user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv("user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv("user_app_actived.csv", names=['uid','appId'])
app_info = pd.read_csv("app_info.csv", names=['appId', 'category'])

In [15]:
#将user_basic_info.csv 和 user_behavior_info.csv中的字符值编码成可以训练的数值类型，合并
class2id = {}
id2class = {}
def mergeBasicTables(baseTable):
    resTable = baseTable.merge(user_basic_info, how='left', on='uid', suffixes=('_base0', '_ubaf'))
    resTable = resTable.merge(user_behavior_info, how='left', on='uid', suffixes=('_base1', '_ubef'))
    cat_columns = ['city','prodName','color','carrier','os','ct']
    for c in cat_columns:
        resTable[c] = resTable[c].apply(lambda x: x if type(x)==str else str(x))
        sort_temp = sorted(list(set(resTable[c])))  
        class2id[c+'2id'] = dict(zip(sort_temp, range(1, len(sort_temp)+1)))
        id2class['id2'+c] = dict(zip(range(1,len(sort_temp)+1), sort_temp))
        resTable[c] = resTable[c].apply(lambda x: class2id[c+'2id'][x])
        
    return resTable

In [16]:
#处理app使用相关数据
#对user_app_actived.csv简单统计
#将之前训练的appuseProcess.csv进行合并

category2id = {'主题个性': 0, '主题铃声': 1, '休闲娱乐': 2, '休闲游戏': 3, 
               '休闲益智': 4, '体育射击': 5, '体育竞速': 6, '便捷生活': 7, 
               '儿童': 8, '出行导航': 9, '动作冒险': 10, '动作射击': 11, 
               '医疗健康': 12, '合作壁纸*': 13, '商务': 14, '图书阅读': 15, 
               '学习办公': 16, '实用工具': 17, '影音娱乐': 18, '拍摄美化': 19, 
               '教育': 20, '新闻阅读': 21, '旅游住宿': 22, '棋牌天地': 23, 
               '棋牌桌游': 24, '模拟游戏': 25, '汽车': 26, '电子书籍': 27, 
               '益智棋牌': 28, '社交通讯': 29, '策略游戏': 30, '经营策略': 31, 
               '网络游戏': 32, '美食': 33, '表盘个性': 34, '角色扮演': 35, 
               '角色游戏': 36, '购物比价': 37, '运动健康': 38, '金融理财': 39}


def mergeAppData(baseTable):
    resTable = baseTable.merge(user_app_actived, how='left', on='uid')
    
    # 将appId列 => 用户对于40类APP是否使用
    temp_app_info = pd.read_csv("app_info.csv", names=['appId','category'])
    dict_temp_app_info = temp_app_info.set_index('appId').T.to_dict()  # 将dataframe转dict
    
    
    def fun(x):
        newList = []
        for i in list(x.split('#')):
            if i in dict_temp_app_info.keys():
                newList.append(category2id[dict_temp_app_info[i]['category']])
            else:
                newList.append(40)
        newList.append(40)        
        return np.bincount(newList)
        
         

    app_category = resTable['appId'].apply(fun)   # 将appId列表转换为APP所属的类别的编号的bincount
    
    app_category_0 = []
    app_category_1 = []
    app_category_2 = []
    app_category_3 = []
    app_category_4 = []
    app_category_5 = []
    app_category_6 = []
    app_category_7 = []
    app_category_8 = []
    app_category_9 = []
    app_category_10 = []
    app_category_11 = []
    app_category_12 = []
    app_category_13 = []
    app_category_14 = []
    app_category_15 = []
    app_category_16 = []
    app_category_17 = []
    app_category_18 = []
    app_category_19 = []
    app_category_20 = []
    app_category_21 = []
    app_category_22 = []
    app_category_23 = []
    app_category_24 = []
    app_category_25 = []
    app_category_26 = []
    app_category_27 = []
    app_category_28 = []
    app_category_29 = []
    app_category_30 = []
    app_category_31 = []
    app_category_32 = []
    app_category_33 = []
    app_category_34 = []
    app_category_35 = []
    app_category_36 = []
    app_category_37 = []
    app_category_38 = []
    app_category_39 = []
    app_category_40 = []
    
    for category in app_category:
        app_category_0.append(category[0])
        app_category_1.append(category[1])
        app_category_2.append(category[2])
        app_category_3.append(category[3])
        app_category_4.append(category[4])
        app_category_5.append(category[5])
        app_category_6.append(category[6])
        app_category_7.append(category[7])
        app_category_8.append(category[8])
        app_category_9.append(category[9])
        app_category_10.append(category[10])
        app_category_11.append(category[11])
        app_category_12.append(category[12])
        app_category_13.append(category[13])
        app_category_14.append(category[14])
        app_category_15.append(category[15])
        app_category_16.append(category[16])
        app_category_17.append(category[17])
        app_category_18.append(category[18])
        app_category_19.append(category[19])
        app_category_20.append(category[20])
        app_category_21.append(category[21])
        app_category_22.append(category[22])
        app_category_23.append(category[23])
        app_category_24.append(category[24])
        app_category_25.append(category[25])
        app_category_26.append(category[26])
        app_category_27.append(category[27])
        app_category_28.append(category[28])
        app_category_29.append(category[29])
        app_category_30.append(category[30])
        app_category_31.append(category[31])
        app_category_32.append(category[32])
        app_category_33.append(category[33])
        app_category_34.append(category[34])
        app_category_35.append(category[35])
        app_category_36.append(category[36])
        app_category_37.append(category[37])
        app_category_38.append(category[38])
        app_category_39.append(category[39])
        app_category_40.append(category[40])
    
    temp_df = pd.DataFrame()
    temp_df['uid'] = resTable['uid']
    
    temp_df.insert(0, 'app_category_40', app_category_40)
    temp_df.insert(0, 'app_category_39', app_category_39)
    temp_df.insert(0, 'app_category_38', app_category_38)
    temp_df.insert(0, 'app_category_37', app_category_37)
    temp_df.insert(0, 'app_category_36', app_category_36)
    temp_df.insert(0, 'app_category_35', app_category_35)
    temp_df.insert(0, 'app_category_34', app_category_34)
    temp_df.insert(0, 'app_category_33', app_category_33)
    temp_df.insert(0, 'app_category_32', app_category_32)
    temp_df.insert(0, 'app_category_31', app_category_31)
    temp_df.insert(0, 'app_category_30', app_category_30)
    temp_df.insert(0, 'app_category_29', app_category_29)
    temp_df.insert(0, 'app_category_28', app_category_28)
    temp_df.insert(0, 'app_category_27', app_category_27)
    temp_df.insert(0, 'app_category_26', app_category_26)
    temp_df.insert(0, 'app_category_25', app_category_25)
    temp_df.insert(0, 'app_category_24', app_category_24)
    temp_df.insert(0, 'app_category_23', app_category_23)
    temp_df.insert(0, 'app_category_22', app_category_22)
    temp_df.insert(0, 'app_category_21', app_category_21)
    temp_df.insert(0, 'app_category_20', app_category_20)
    temp_df.insert(0, 'app_category_19', app_category_19)
    temp_df.insert(0, 'app_category_18', app_category_18)
    temp_df.insert(0, 'app_category_17', app_category_17)
    temp_df.insert(0, 'app_category_16', app_category_16)
    temp_df.insert(0, 'app_category_15', app_category_15)
    temp_df.insert(0, 'app_category_14', app_category_14)
    temp_df.insert(0, 'app_category_13', app_category_13)
    temp_df.insert(0, 'app_category_12', app_category_12)
    temp_df.insert(0, 'app_category_11', app_category_11)
    temp_df.insert(0, 'app_category_10', app_category_10)
    temp_df.insert(0, 'app_category_9', app_category_9)
    temp_df.insert(0, 'app_category_8', app_category_8)
    temp_df.insert(0, 'app_category_7', app_category_7)
    temp_df.insert(0, 'app_category_6', app_category_6)
    temp_df.insert(0, 'app_category_5', app_category_5)
    temp_df.insert(0, 'app_category_4', app_category_4)
    temp_df.insert(0, 'app_category_3', app_category_3)
    temp_df.insert(0, 'app_category_2', app_category_2)
    temp_df.insert(0, 'app_category_1', app_category_1)
    temp_df.insert(0, 'app_category_0', app_category_0)
    
    resTable = resTable.merge(temp_df, how='left', on='uid')
    
    del temp_df
    del app_category_0
    del app_category_1 
    del app_category_2 
    del app_category_3 
    del app_category_4 
    del app_category_5 
    del app_category_6 
    del app_category_7 
    del app_category_8 
    del app_category_9 
    del app_category_10
    del app_category_11
    del app_category_12
    del app_category_13 
    del app_category_14 
    del app_category_15
    del app_category_16 
    del app_category_17
    del app_category_18
    del app_category_19 
    del app_category_20 
    del app_category_21
    del app_category_22 
    del app_category_23 
    del app_category_24 
    del app_category_25
    del app_category_26 
    del app_category_27
    del app_category_28 
    del app_category_29
    del app_category_30 
    del app_category_31 
    del app_category_32
    del app_category_33 
    del app_category_34
    del app_category_35 
    del app_category_36
    del app_category_37 
    del app_category_38 
    del app_category_39 
    del app_category_40 
     
    appusedTable = pd.read_csv("appuseProcessed.csv")
    resTable = resTable.merge(appusedTable, how='left', on='uid')
    resTable[['category', 'usage_most_used_category']] = resTable[['category', 'usage_most_used_category']].fillna(41)
    resTable = resTable.fillna(0)
    return resTable

In [17]:
#合并用户基本特征以及app使用相关特征，作为训练集和测试集
df_train = mergeAppData(mergeBasicTables(age_train))
df_test = mergeAppData(mergeBasicTables(age_test))

del df_train['appId']
del df_test['appId']

del age_train
del age_test


d:\workenvironment\python3.7\lib\site-packages\ipykernel_launcher.py:22: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [18]:
# 41类APP的使用次数
app_category_time = pd.read_csv("app_category_time.csv",names=['uid','app_timeId_0','app_timeId_1','app_timeId_2','app_timeId_3','app_timeId_4','app_timeId_5','app_timeId_6','app_timeId_7','app_timeId_8','app_timeId_9','app_timeId_10','app_timeId_11','app_timeId_12','app_timeId_13','app_timeId_14','app_timeId_15','app_timeId_16','app_timeId_17','app_timeId_18','app_timeId_19','app_timeId_20','app_timeId_21','app_timeId_22','app_timeId_23','app_timeId_24','app_timeId_25','app_timeId_26','app_timeId_27','app_timeId_28','app_timeId_29','app_timeId_30','app_timeId_31','app_timeId_32','app_timeId_33','app_timeId_34','app_timeId_35','app_timeId_36','app_timeId_37','app_timeId_38','app_timeId_39','app_timeId_40'])

app_category_time = app_category_time.groupby('uid').sum().reset_index()

In [19]:
df_train = df_train.merge(app_category_time, how='left', on='uid')
df_test = df_test.merge(app_category_time, how='left', on='uid')

df_train.fillna(0, inplace = True)
df_test.fillna(0, inplace = True)

del app_category_time

In [20]:
# 41类APP的使用时长
app_category_duration = pd.read_csv("app_category_duration.csv",names=['uid','app_categoryId_0','app_categoryId_1','app_categoryId_2','app_categoryId_3','app_categoryId_4','app_categoryId_5','app_categoryId_6','app_categoryId_7','app_categoryId_8','app_categoryId_9','app_categoryId_10','app_categoryId_11','app_categoryId_12','app_categoryId_13','app_categoryId_14','app_categoryId_15','app_categoryId_16','app_categoryId_17','app_categoryId_18','app_categoryId_19','app_categoryId_20','app_categoryId_21','app_categoryId_22','app_categoryId_23','app_categoryId_24','app_categoryId_25','app_categoryId_26','app_categoryId_27','app_categoryId_28','app_categoryId_29','app_categoryId_30','app_categoryId_31','app_categoryId_32','app_categoryId_33','app_categoryId_34','app_categoryId_35','app_categoryId_36','app_categoryId_37','app_categoryId_38','app_categoryId_39','app_categoryId_40'])
app_category_duration = app_category_duration.groupby('uid').sum().reset_index()

In [21]:
df_train = df_train.merge(app_category_duration, how='left', on='uid')
df_test = df_test.merge(app_category_duration, how='left', on='uid')

df_train.fillna(0, inplace = True)
df_test.fillna(0, inplace = True)

del app_category_duration

In [22]:
print(df_test)

            uid  gender  city  prodName  ramCapacity  ramLeftRation  \
0       1000002       1   298       113          0.0           0.26   
1       1000003       0   222         6          8.0           0.35   
2       1000004       1   198        31          3.0           0.36   
3       1000005       0   107         1          6.0           0.00   
4       1000006       0   284       146          2.0           0.19   
5       1000007       1   314       191          6.0           0.20   
6       1000008       1   298        52          6.0           0.36   
7       1000009       0   336        52          6.0           0.39   
8       1000010       0   298       128          6.0           0.36   
9       1000012       0   201        66          4.0           0.31   
10      1000013       0   133        50          4.0           0.49   
11      1000014       1   314         3          6.0           0.49   
12      1000016       0   348       130          3.0           0.41   
13    

[502500 rows x 155 columns]


In [13]:
X = df_train.drop(['age_group', 'uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=80)
X_new = pca.fit_transform(X)
test_new = pca.fit_transform(test)

del X
del test

In [14]:
#训练模型

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [16]:
print("训练模型：")
param = {
        'learning_rate': 0.1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.8,
        'metric': 'multi_error'
}


xx_score = []
cv_pred = []  
skf = StratifiedKFold(n_splits=3, random_state=1030, shuffle=True)      # 分层采样3次
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):     # enumerate同时列出索引和值, tqdm显示进度
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=2000, valid_sets=[vali], early_stopping_rounds=50)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))  # hstack将array在水平方向上拼接
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))   # 投票

训练模型：
0
training start...
[1]	valid_0's multi_error: 0.701493
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.69192
[3]	valid_0's multi_error: 0.667435
[4]	valid_0's multi_error: 0.648312
[5]	valid_0's multi_error: 0.634054
[6]	valid_0's multi_error: 0.618624
[7]	valid_0's multi_error: 0.603523
[8]	valid_0's multi_error: 0.593798
[9]	valid_0's multi_error: 0.584219
[10]	valid_0's multi_error: 0.577108
[11]	valid_0's multi_error: 0.572486
[12]	valid_0's multi_error: 0.568262
[13]	valid_0's multi_error: 0.564756
[14]	valid_0's multi_error: 0.561422
[15]	valid_0's multi_error: 0.558723
[16]	valid_0's multi_error: 0.55659
[17]	valid_0's multi_error: 0.553889
[18]	valid_0's multi_error: 0.552105
[19]	valid_0's multi_error: 0.549775
[20]	valid_0's multi_error: 0.547963
[21]	valid_0's multi_error: 0.546265
[22]	valid_0's multi_error: 0.544714
[23]	valid_0's multi_error: 0.543305
[24]	valid_0's multi_error: 0.541925
[25]	valid_0's multi_error: 0.54050

[433]	valid_0's multi_error: 0.470243
[434]	valid_0's multi_error: 0.470205
[435]	valid_0's multi_error: 0.470114
[436]	valid_0's multi_error: 0.470143
[437]	valid_0's multi_error: 0.470147
[438]	valid_0's multi_error: 0.47011
[439]	valid_0's multi_error: 0.470107
[440]	valid_0's multi_error: 0.470123
[441]	valid_0's multi_error: 0.470071
[442]	valid_0's multi_error: 0.470077
[443]	valid_0's multi_error: 0.470083
[444]	valid_0's multi_error: 0.470065
[445]	valid_0's multi_error: 0.470017
[446]	valid_0's multi_error: 0.470002
[447]	valid_0's multi_error: 0.469993
[448]	valid_0's multi_error: 0.469952
[449]	valid_0's multi_error: 0.46991
[450]	valid_0's multi_error: 0.469884
[451]	valid_0's multi_error: 0.469834
[452]	valid_0's multi_error: 0.469822
[453]	valid_0's multi_error: 0.469769
[454]	valid_0's multi_error: 0.469762
[455]	valid_0's multi_error: 0.469716
[456]	valid_0's multi_error: 0.469687
[457]	valid_0's multi_error: 0.469647
[458]	valid_0's multi_error: 0.469695
[459]	valid_0'

[865]	valid_0's multi_error: 0.465322
[866]	valid_0's multi_error: 0.465326
[867]	valid_0's multi_error: 0.465331
[868]	valid_0's multi_error: 0.465329
[869]	valid_0's multi_error: 0.465337
[870]	valid_0's multi_error: 0.46532
[871]	valid_0's multi_error: 0.465341
[872]	valid_0's multi_error: 0.46538
[873]	valid_0's multi_error: 0.465398
[874]	valid_0's multi_error: 0.465365
[875]	valid_0's multi_error: 0.465363
[876]	valid_0's multi_error: 0.465355
[877]	valid_0's multi_error: 0.46538
[878]	valid_0's multi_error: 0.465363
[879]	valid_0's multi_error: 0.465368
[880]	valid_0's multi_error: 0.465358
[881]	valid_0's multi_error: 0.465284
[882]	valid_0's multi_error: 0.465271
[883]	valid_0's multi_error: 0.465295
[884]	valid_0's multi_error: 0.465274
[885]	valid_0's multi_error: 0.465269
[886]	valid_0's multi_error: 0.465225
[887]	valid_0's multi_error: 0.465205
[888]	valid_0's multi_error: 0.465183
[889]	valid_0's multi_error: 0.465175
[890]	valid_0's multi_error: 0.465172
[891]	valid_0's

[1289]	valid_0's multi_error: 0.46322
[1290]	valid_0's multi_error: 0.463183
[1291]	valid_0's multi_error: 0.463168
[1292]	valid_0's multi_error: 0.463163
[1293]	valid_0's multi_error: 0.463149
[1294]	valid_0's multi_error: 0.46318
[1295]	valid_0's multi_error: 0.463183
[1296]	valid_0's multi_error: 0.46318
[1297]	valid_0's multi_error: 0.463169
[1298]	valid_0's multi_error: 0.463195
[1299]	valid_0's multi_error: 0.463165
[1300]	valid_0's multi_error: 0.463131
[1301]	valid_0's multi_error: 0.463144
[1302]	valid_0's multi_error: 0.463129
[1303]	valid_0's multi_error: 0.463117
[1304]	valid_0's multi_error: 0.463146
[1305]	valid_0's multi_error: 0.463144
[1306]	valid_0's multi_error: 0.463174
[1307]	valid_0's multi_error: 0.463159
[1308]	valid_0's multi_error: 0.463161
[1309]	valid_0's multi_error: 0.463177
[1310]	valid_0's multi_error: 0.463137
[1311]	valid_0's multi_error: 0.463129
[1312]	valid_0's multi_error: 0.463168
[1313]	valid_0's multi_error: 0.463149
[1314]	valid_0's multi_error

[1709]	valid_0's multi_error: 0.462183
[1710]	valid_0's multi_error: 0.462162
[1711]	valid_0's multi_error: 0.462158
[1712]	valid_0's multi_error: 0.462149
[1713]	valid_0's multi_error: 0.462125
[1714]	valid_0's multi_error: 0.462149
[1715]	valid_0's multi_error: 0.462152
[1716]	valid_0's multi_error: 0.462163
[1717]	valid_0's multi_error: 0.462178
[1718]	valid_0's multi_error: 0.462183
[1719]	valid_0's multi_error: 0.462181
[1720]	valid_0's multi_error: 0.462152
[1721]	valid_0's multi_error: 0.462152
[1722]	valid_0's multi_error: 0.462165
[1723]	valid_0's multi_error: 0.462155
[1724]	valid_0's multi_error: 0.462175
[1725]	valid_0's multi_error: 0.462169
[1726]	valid_0's multi_error: 0.462168
[1727]	valid_0's multi_error: 0.462129
[1728]	valid_0's multi_error: 0.462116
[1729]	valid_0's multi_error: 0.462128
[1730]	valid_0's multi_error: 0.462071
[1731]	valid_0's multi_error: 0.462083
[1732]	valid_0's multi_error: 0.462074
[1733]	valid_0's multi_error: 0.46208
[1734]	valid_0's multi_err

...

[1163]	valid_0's multi_error: 0.462466
[1164]	valid_0's multi_error: 0.462464
[1165]	valid_0's multi_error: 0.462442
[1166]	valid_0's multi_error: 0.462472
[1167]	valid_0's multi_error: 0.462455
[1168]	valid_0's multi_error: 0.462419
[1169]	valid_0's multi_error: 0.462407
[1170]	valid_0's multi_error: 0.462419
[1171]	valid_0's multi_error: 0.462451
[1172]	valid_0's multi_error: 0.462443
[1173]	valid_0's multi_error: 0.462436
[1174]	valid_0's multi_error: 0.462427
[1175]	valid_0's multi_error: 0.462412
[1176]	valid_0's multi_error: 0.462394
[1177]	valid_0's multi_error: 0.462348
[1178]	valid_0's multi_error: 0.462331
[1179]	valid_0's multi_error: 0.462316
[1180]	valid_0's multi_error: 0.462336
[1181]	valid_0's multi_error: 0.462331
[1182]	valid_0's multi_error: 0.462349
[1183]	valid_0's multi_error: 0.462348
[1184]	valid_0's multi_error: 0.462369
[1185]	valid_0's multi_error: 0.462346
[1186]	valid_0's multi_error: 0.462357
[1187]	valid_0's multi_error: 0.462337
[1188]	valid_0's multi_er

[125]	valid_0's multi_error: 0.493937
[126]	valid_0's multi_error: 0.493562
[127]	valid_0's multi_error: 0.493371
[128]	valid_0's multi_error: 0.493135
[129]	valid_0's multi_error: 0.492917
[130]	valid_0's multi_error: 0.492716
[131]	valid_0's multi_error: 0.492581
[132]	valid_0's multi_error: 0.492398
[133]	valid_0's multi_error: 0.492231
[134]	valid_0's multi_error: 0.492092
[135]	valid_0's multi_error: 0.491914
[136]	valid_0's multi_error: 0.49164
[137]	valid_0's multi_error: 0.49149
[138]	valid_0's multi_error: 0.491348
[139]	valid_0's multi_error: 0.491146
[140]	valid_0's multi_error: 0.490971
[141]	valid_0's multi_error: 0.490743
[142]	valid_0's multi_error: 0.490656
[143]	valid_0's multi_error: 0.490353
[144]	valid_0's multi_error: 0.490278
[145]	valid_0's multi_error: 0.490066
[146]	valid_0's multi_error: 0.489916
[147]	valid_0's multi_error: 0.489846
[148]	valid_0's multi_error: 0.489762
[149]	valid_0's multi_error: 0.489557
[150]	valid_0's multi_error: 0.48944
[151]	valid_0's

[557]	valid_0's multi_error: 0.467822
[558]	valid_0's multi_error: 0.467778
[559]	valid_0's multi_error: 0.46774
[560]	valid_0's multi_error: 0.467748
[561]	valid_0's multi_error: 0.467734
[562]	valid_0's multi_error: 0.467747
[563]	valid_0's multi_error: 0.467711
[564]	valid_0's multi_error: 0.467737
[565]	valid_0's multi_error: 0.467722
[566]	valid_0's multi_error: 0.467675
[567]	valid_0's multi_error: 0.467683
[568]	valid_0's multi_error: 0.467656
[569]	valid_0's multi_error: 0.467623
[570]	valid_0's multi_error: 0.467631
[571]	valid_0's multi_error: 0.467678
[572]	valid_0's multi_error: 0.467626
[573]	valid_0's multi_error: 0.467625
[574]	valid_0's multi_error: 0.467584
[575]	valid_0's multi_error: 0.467554
[576]	valid_0's multi_error: 0.467568
[577]	valid_0's multi_error: 0.467525
[578]	valid_0's multi_error: 0.467534
[579]	valid_0's multi_error: 0.467563
[580]	valid_0's multi_error: 0.467563
[581]	valid_0's multi_error: 0.467548
[582]	valid_0's multi_error: 0.467545
[583]	valid_0

[989]	valid_0's multi_error: 0.463993
[990]	valid_0's multi_error: 0.464013
[991]	valid_0's multi_error: 0.464047
[992]	valid_0's multi_error: 0.464071
[993]	valid_0's multi_error: 0.464102
[994]	valid_0's multi_error: 0.464096
[995]	valid_0's multi_error: 0.46408
[996]	valid_0's multi_error: 0.464075
[997]	valid_0's multi_error: 0.464048
[998]	valid_0's multi_error: 0.464029
[999]	valid_0's multi_error: 0.463969
[1000]	valid_0's multi_error: 0.463998
[1001]	valid_0's multi_error: 0.464016
[1002]	valid_0's multi_error: 0.463993
[1003]	valid_0's multi_error: 0.463984
[1004]	valid_0's multi_error: 0.463977
[1005]	valid_0's multi_error: 0.463989
[1006]	valid_0's multi_error: 0.463956
[1007]	valid_0's multi_error: 0.463914
[1008]	valid_0's multi_error: 0.463934
[1009]	valid_0's multi_error: 0.463931
[1010]	valid_0's multi_error: 0.463929
[1011]	valid_0's multi_error: 0.463898
[1012]	valid_0's multi_error: 0.463892
[1013]	valid_0's multi_error: 0.463813
[1014]	valid_0's multi_error: 0.46379

[1409]	valid_0's multi_error: 0.462339
[1410]	valid_0's multi_error: 0.462342
[1411]	valid_0's multi_error: 0.462351
[1412]	valid_0's multi_error: 0.462368
[1413]	valid_0's multi_error: 0.462348
[1414]	valid_0's multi_error: 0.462325
[1415]	valid_0's multi_error: 0.462305
[1416]	valid_0's multi_error: 0.462268
[1417]	valid_0's multi_error: 0.462272
[1418]	valid_0's multi_error: 0.462296
[1419]	valid_0's multi_error: 0.462323
[1420]	valid_0's multi_error: 0.462325
[1421]	valid_0's multi_error: 0.462313
[1422]	valid_0's multi_error: 0.46232
[1423]	valid_0's multi_error: 0.462332
[1424]	valid_0's multi_error: 0.462319
[1425]	valid_0's multi_error: 0.462301
[1426]	valid_0's multi_error: 0.462268
[1427]	valid_0's multi_error: 0.462275
[1428]	valid_0's multi_error: 0.462263
[1429]	valid_0's multi_error: 0.462274
[1430]	valid_0's multi_error: 0.462257
[1431]	valid_0's multi_error: 0.462259
[1432]	valid_0's multi_error: 0.462262
[1433]	valid_0's multi_error: 0.462301
[1434]	valid_0's multi_err

In [17]:
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})

print(df)

df.to_csv('submission.csv',index=False)

d:\workenvironment\python3.7\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


             id  label
0       1000002      4
1       1000003      5
2       1000004      3
3       1000005      2
4       1000006      3
5       1000007      3
6       1000008      3
7       1000009      3
8       1000010      4
9       1000012      6
10      1000013      4
11      1000014      5
12      1000016      5
13      1000017      3
14      1000018      2
15      1000020      5
16      1000021      3
17      1000022      5
18      1000024      3
19      1000026      6
20      1000027      4
21      1000028      4
22      1000030      3
23      1000031      3
24      1000032      3
25      1000033      4
26      1000034      5
27      1000036      4
28      1000039      4
29      1000041      5
...         ...    ...
502470  3178805      3
502471  3179045      1
502472  3179064      2
502473  3179100      2
502474  3179128      3
502475  3179165      2
502476  3179213      2
502477  3179270      2
502478  3179327      1
502479  3179417      3
502480  3179418      2
502481  317